In [ ]:
from surianalytics.connectors import RESTSciriusConnector

In [ ]:
from surianalytics.widgets.config import update_connector_time_delta

In [ ]:
CONNECTOR = RESTSciriusConnector()

In [ ]:
CONNECTOR.set_page_size(10000)

In [ ]:
update_connector_time_delta(c=CONNECTOR, dump_path="../config.json")

In [ ]:
from IPython.display import JSON

In [ ]:
import pandas as pd

In [ ]:
import ipywidgets as widgets

In [ ]:
DATA = CONNECTOR.get_eve_unique_values(qfilter="event_type: smb", 
                                    field="flow_id", 
                                    counts="yes", 
                                    size=100)

In [ ]:
pd.DataFrame(DATA)

In [ ]:
import ipywidgets as widgets

In [ ]:
import pandas as pd

In [ ]:
from ipywidgets.widgets.interaction import display

In [ ]:
def show(limit: int,
         fid: int):
    pd.set_option('display.max_rows', limit)
    pd.set_option('display.min_rows', limit)
    pd.set_option('display.max_columns', None)
    
    df = (
        CONNECTOR
        .get_events_df(qfilter="flow_id: {fid}".format(fid=fid))
    )
    
    cols = ["host", "timestamp", "flow_id", "src_ip", "dest_ip", "src_port", "dest_port"]
    
    for col in ["alert.signature"]:
        if col in list(df.columns.values):
            cols = cols + [col]
            
    cols += [c for c in list(df.columns.values) if c.startswith("smb.")]

    display(df.sort_values(by=["timestamp"])[cols])

In [ ]:
widgets.interact(show, 
                 limit=widgets.IntSlider(min=10, max=1000),
                 fid=widgets.Dropdown(options=[f["key"] for f in DATA]))

In [ ]:
DF = pd.DataFrame()
for fid in [d["key"] for d in DATA]:
    df = (
        CONNECTOR
        .get_events_df(qfilter="flow_id: {fid}".format(fid=fid))
    )
    DF = pd.concat([DF, df], axis=0)

In [ ]:
pd.options.display.html.use_mathjax = False

In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.min_rows', 50)

In [ ]:
pd.set_option('max_colwidth', 1000)

In [ ]:
DF.groupby("src_ip").agg({
    "src_ip": ["unique", "nunique"],
    "smb.command": ["unique", "nunique"],
    "smb.status": ["unique", "nunique"],
    # "smb.ntlmssp.user": ["unique", "nunique"],
    "smb.filename": ["unique", "nunique"],
    "smb.named_pipe": ["unique", "nunique"],
    "alert.signature": ["unique", "nunique"],
    "timestamp": ["min", "max"]
}).sort_values(by=[('src_ip', 'nunique')], ascending=False)